# Block descent for optimizing group knockoffs

There are 2 ways for optimizing the group knockoff problem. This notebook tests the block update strategy where we update an entire block at once. Two concerns arise 

+ Which solver (MOSEK, Hypatia.jl, SCS, SDPT3, Convex.jl) should we use to solve each individual block?
+ How to maintain inequality constraints after updating a block? Can we use Woodbury?

In [54]:
using Revise
using Knockoffs
using BlockDiagonals
using LinearAlgebra
using Hypatia
using JuMP
using ToeplitzMatrices
using StatsBase
using CovarianceEstimation
using CSV, DataFrames
using Ipopt, NLopt, Convex
# ENV["COLUMNS"] = 240

#
# test problem
#
# p = 50
# m = 5
# group_sizes = [5 for i in 1:div(p, 5)] # each group has 5 variables
# groups = vcat([i*ones(g) for (i, g) in enumerate(group_sizes)]...) |> Vector{Int}
# x = randn(p, p)
# Σ = x'*x
# StatsBase.cov2cor!(Σ, std(Σ, dims=1))

# genomAD panel
datadir = "/Users/biona001/Benjamin_Folder/research/4th_project_PRS/group_knockoff_test_data"
covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
Σ = covfile |> Matrix{Float64}
Σ = 0.99Σ + 0.01I #ensure PSD
groups, group_reps = hc_partition_groups(Symmetric(Σ))

# test on smaller data
p = 100
m = 5
groups = repeat(1:div(p, 5), inner=5)
Σ = Σ[1:p, 1:p];

## Suboptimal SDP

In [130]:
@time Gsdp_subopt, _, obj = solve_s_group(
    Symmetric(Σ), 
    groups, 
    :sdp_subopt,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

  1.261893 seconds (1.11 M allocations: 63.849 MiB, 1.77% gc time, 21.34% compilation time)


In [131]:
sum(abs.(Gsdp_subopt))

2.720124164272983

In [132]:
eigmin((m+1)/m*Σ - Gsdp_subopt), eigmin(Gsdp_subopt)

(-2.4252546212458004e-10, 4.428711116847948e-5)

## Block SDP

In [126]:
@time Gsdp, _, obj = solve_s_group(
    Symmetric(Σ),
    groups,
    :sdp_block,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

Init obj = 2868.7908916672664
Iter 1 δ = 0.9929532250810528, obj = 2857.810644184678
Iter 2 δ = 0.6221453499032936, obj = 2854.1597179031833
Iter 3 δ = 0.13130461806607607, obj = 2853.4995363108455
Iter 4 δ = 0.03834661429018987, obj = 2853.314467480943
Iter 5 δ = 0.04058967366268737, obj = 2853.193056709868
Iter 6 δ = 0.03049073970297339, obj = 2853.1130065961615
Iter 7 δ = 0.030936162665849976, obj = 2853.0381325798444
Iter 8 δ = 0.029283747956588613, obj = 2852.952185013389
Iter 9 δ = 0.03438359235629401, obj = 2852.861722950144
Iter 10 δ = 0.07124590913561568, obj = 2852.7275517399357
Iter 11 δ = 0.02736142060737984, obj = 2852.6580675933083
Iter 12 δ = 0.016888691523540555, obj = 2852.6051696479285
Iter 13 δ = 0.03436599157056314, obj = 2852.541328269604
Iter 14 δ = 0.024119637372275238, obj = 2852.4891425792885
Iter 15 δ = 0.016335856139469973, obj = 2852.4457585991477
Iter 16 δ = 0.01158223593463012, obj = 2852.4283080180867
Iter 17 δ = 0.009336714549595282, obj = 2852.407111105

In [127]:
eigmin(Gsdp), eigmin((m+1)/m*Σ - Gsdp)

(-1.2546813406697078e-9, -9.994414773150977e-6)

In [128]:
sum(abs.(Gsdp))

19.44089460318265

In [135]:
@time Gsdp, _, obj = solve_s_group(
    Symmetric(Σ),
    groups,
    :sdp,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

solve_group_SDP_ccd: Optimizing 500 variables
initial obj = 2868.7907916672716
Iter 1: obj = 2866.3416724918457, δ = 0.9929522292643561, t1 = 0.0, t2 = 0.0, t3 = 0.0
Iter 2: obj = 2865.6570471534505, δ = 0.26295497223363856, t1 = 0.0, t2 = 0.0, t3 = 0.0
Iter 3: obj = 2864.806287833113, δ = 0.4764822976002814, t1 = 0.0, t2 = 0.01, t3 = 0.0
Iter 4: obj = 2864.601567346938, δ = 0.08392543485250858, t1 = 0.01, t2 = 0.01, t3 = 0.0
Iter 5: obj = 2864.4990360795564, δ = 0.03807226282857241, t1 = 0.01, t2 = 0.01, t3 = 0.0
Iter 6: obj = 2864.401582440762, δ = 0.059758674084312007, t1 = 0.01, t2 = 0.01, t3 = 0.0
Iter 7: obj = 2864.29947600357, δ = 0.06679331013702434, t1 = 0.01, t2 = 0.01, t3 = 0.0
Iter 8: obj = 2864.189951575968, δ = 0.06763964224223061, t1 = 0.01, t2 = 0.01, t3 = 0.0
Iter 9: obj = 2863.840970607509, δ = 0.2175536307523194, t1 = 0.01, t2 = 0.02, t3 = 0.0
Iter 10: obj = 2863.690005459616, δ = 0.05645209379556937, t1 = 0.01, t2 = 0.02, t3 = 0.0
Iter 11: obj = 2863.4678730212904, 

In [136]:
eigmin(Gsdp), eigmin((m+1)/m*Σ - Gsdp)

(2.0532606812920924e-7, -1.9999151938430062e-6)

In [137]:
sum(abs.(Gsdp))

7.795120347420987

## Block ME

In [138]:
@time Gme, _, obj = solve_s_group(
    Symmetric(Σ),
    groups,
    :maxent_block,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

Init obj = -3240.198615576776
Iter 1 δ = 0.5850885752184042, obj = -2051.814882669756
Iter 2 δ = 0.025704025773012687, obj = -2037.956381647256
Iter 3 δ = 8.967094959807728e-6, obj = -2037.9511883873668
  7.965265 seconds (2.12 M allocations: 166.803 MiB, 1.04% gc time)


In [139]:
eigmin(Gme), eigmin((m+1)/m*Σ - Gme)

(0.005999999895569638, 0.004378692585123525)

In [140]:
sum(abs.(Gme))

14.193654813870936

In [141]:
@time Gme, _, obj = solve_s_group(
    Symmetric(Σ),
    groups,
    :maxent,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

solve_group_max_entropy_ccd: Optimizing 500 variables
initial obj = -3219.795159204885
Iter 1: obj = -2245.925785923409, δ = 0.5851610065569198, t1 = 0.0, t2 = 0.0, t3 = 0.0
Iter 2: obj = -2177.9317767708308, δ = 0.05064598674328402, t1 = 0.0, t2 = 0.0, t3 = 0.0
Iter 3: obj = -2123.8950743424957, δ = 0.00681211972618978, t1 = 0.01, t2 = 0.01, t3 = 0.0
  0.019249 seconds (2.28 k allocations: 1.781 MiB)


In [142]:
eigmin(Gme), eigmin((m+1)/m*Σ - Gme)

(0.0005391777812456451, 0.001585899760471455)

In [143]:
sum(abs.(Gme))

12.487420100051178

## Block MVR

In [120]:
@time Gmvr, _, obj = solve_s_group(
    Symmetric(Σ),
    groups,
    :mvr_block,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

Init obj = 8.024310739326389e13
Iter 1 δ = 0.9529062400705861, obj = 20526.417700374473
Iter 2 δ = 0.30168802059049393, obj = 19601.551832992256
Iter 3 δ = 0.010826254405392866, obj = 19599.800666254127
Iter 4 δ = 0.0012768638185918224, obj = 19599.794824824377
632.825699 seconds (29.95 M allocations: 5.004 GiB, 0.07% gc time)


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/qzdJ6/src/Solvers/steppers/combined.jl:111
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/

In [121]:
eigmin(Gmvr), eigmin((m+1)/m*Σ - Gmvr)

(7.046770735640549e-5, 0.0021349946279600874)

In [122]:
sum(abs.(Gmvr))

16.775770215404926

In [144]:
@time Gmvr, _, obj = solve_s_group(
    Symmetric(Σ),
    groups,
    :mvr,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

solve_group_MVR_ccd: Optimizing 500 variables
initial obj = 6.35770633661056e6
Iter 1: obj = 5.05443776455565e6, δ = 0.8246655912286674, t1 = 0.0, t2 = 0.01, t3 = 0.0
Iter 2: obj = 7.13787560092228e6, δ = 0.39426250495128384, t1 = 0.0, t2 = 0.01, t3 = 0.0
Iter 3: obj = 1.3118774116832506e7, δ = 0.10921030264233643, t1 = 0.01, t2 = 0.02, t3 = 0.0
Iter 4: obj = 3.1257590061881166e7, δ = 0.06054720401372597, t1 = 0.01, t2 = 0.03, t3 = 0.0
Iter 5: obj = 8.419118559491296e7, δ = 0.02108859185248942, t1 = 0.01, t2 = 0.03, t3 = 0.0
Iter 6: obj = 3.76700768436183e8, δ = 0.007935252276791155, t1 = 0.01, t2 = 0.04, t3 = 0.0
  0.452877 seconds (6.55 k allocations: 73.281 MiB)


In [145]:
eigmin(Gmvr), eigmin((m+1)/m*Σ - Gmvr)

(9.543150982427232e-8, 0.0001807356077073001)

In [146]:
sum(abs.(Gmvr))

15.869731973406134

In [16]:
@time Gmvr, _ = solve_s_group(
    Symmetric(Σ),
    groups,
    :mvr_block,
    tol = 0.01,
    niter = 100,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

Init obj = 235.27079322796348
S11 = [0.007046770735643412 4.95935464236245e-5 0.002541416243479922 -0.0008206366403779558 0.0008565360371055414; 4.95935464236245e-5 0.007046770735643412 0.0005102341958575869 -0.0003016097743471027 0.003507235099105291; 0.002541416243479922 0.0005102341958575869 0.007046770735643405 -0.0005285326151908768 0.0017629820625260527; -0.0008206366403779558 -0.0003016097743471027 -0.0005285326151908768 0.007046770735643412 -0.00034017149211829805; 0.0008565360371055414 0.003507235099105291 0.0017629820625260527 -0.00034017149211829805 0.007046770735643384]
S11_new = [0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
S11 = [0.007046770735643412 0.0069640381263673745 -0.0003777470123123818 -0.002534082705419375 -0.0025080692277493362; 0.0069640381263673745 0.007046770735643412 -0.00037907206709822497 -0.002527963558553914 -0.002501998587816768; -0.0003777470123123818 -0.00037907206709822497 0.007046770735643

In [12]:
eigmin(Gmvr), eigmin((m+1)/m*Σ - Gmvr)

(7.046770735643238e-5, 2.2004715440131454e-15)

In [13]:
sum(abs.(Gmvr))

1.669665087749101

In [14]:
Gmvr

100×100 Matrix{Float64}:
  0.00704677    4.95935e-5    0.00254142   …   0.0           0.0
  4.95935e-5    0.00704677    0.000510234      0.0           0.0
  0.00254142    0.000510234   0.00704677       0.0           0.0
 -0.000820637  -0.00030161   -0.000528533      0.0           0.0
  0.000856536   0.00350724    0.00176298       0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  ⋮                                        ⋱                
  0.0           0.0           0.0              0.0           0.0
  0.

In [16]:
function solve_group_MVR_single_block(
    Σ11::AbstractMatrix,
    A11::AbstractMatrix, # A = (m+1)/m * Σ
    A12::AbstractMatrix, 
    invD22::AbstractMatrix, # D = A - S
    m::Int,
    ub::AbstractMatrix; # this is upper bound, equals [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false, iter_limit=100) # Any solver compatible with JuMP
    )
    # needed constants
    invD22_A21 = invD22 * A12'
    A12_invD22_A21 = A12 * invD22_A21
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # slack variable to handle tr(inv(S)) terms
    @variable(model, t)
    # objective
#     @constraint(model, t ≥ m^2*tr(inv(S)))
    @NLobjective(model, Min, m^2*tr(inv(S)))
#     @NLobjective(model, Min, 
#         m^2*tr(inv(S)) + tr(inv(A-S-A12_invD22_A21)) + 
#         tr(invD22_A21 * inv(A11-S-A12_invD22_A21) * invD22_A21')
#     )
    # solve and return
    JuMP.optimize!(model)
#     success = check_model_solution(model)
    return JuMP.value.(S), success
end

m = 1
A = (m+1)/m * Σ
Sinit = zeros(p, p)
D = A - Sinit

Σ11 = Σ[1:5, 1:5]
A11 = A[1:5, 1:5]
A12 = A[1:5, 6:end]
D22 = D[6:end, 6:end]
D12 = D[1:5, 6:end]
D21 = D[6:end, 1:5]
invD22 = inv(D22 + 0.00001I)
ub = Symmetric(A11 - D12 * invD22 * D21)

S, _ = solve_group_MVR_single_block(Σ11, A11, A12, invD22, m, ub)

┌ Warning: Function tr automatically registered with 1 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :tr, 1, tr; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear /Users/biona001/.julia/packages/MathOptInterface/cl3eR/src/Nonlinear/operators.jl:370


LoadError: Unexpected array VariableRef[S[1,1] S[1,2] S[1,3] S[1,4] S[1,5]; S[1,2] S[2,2] S[2,3] S[2,4] S[2,5]; S[1,3] S[2,3] S[3,3] S[3,4] S[3,5]; S[1,4] S[2,4] S[3,4] S[4,4] S[4,5]; S[1,5] S[2,5] S[3,5] S[4,5] S[5,5]] in nonlinear expression. Nonlinear expressions may contain only scalar expressions.

In [11]:
eigvals(S)

5-element Vector{Float64}:
 -3.0982246983643755e-12
 -1.8179165550514125e-12
  4.113507165982667e-12
  1.0199610115260541e-11
  3.483242213860298e-11

In [12]:
tr(S)

4.4229398166430407e-11

In [47]:
    invZ_Yt = invD22 * A12'
    Y_invZYt = A12 * invZYt

5×5 Matrix{Float64}:
  0.310498      0.000110537   0.0488194   0.0887203   -0.0467877
  0.000110537   0.374977     -0.100284    0.0936878   -0.0477473
  0.0488194    -0.100284      0.364384   -0.0444843    0.0942238
  0.0887203     0.0936878    -0.0444843   0.358517     0.00535412
 -0.0467877    -0.0477473     0.0942238   0.00535412   0.384187

## discourse

Minimize $tr(S^{-1}) + tr((A - S)^{-1})$ such that $S \succeq 0$ and $A - S \succeq 0$.

In [8]:
using Hypatia, JuMP, LinearAlgebra
p = 5
x = randn(p, p)
A = Symmetric(x' * x)

# model and constraints constraints
model = Model(() -> Hypatia.Optimizer())
@variable(model, S[1:p, 1:p], Symmetric)
@constraint(model, S in PSDCone())
@constraint(model, A - S in PSDCone())

# objective
@NLobjective(model, Min, tr(inv(S)) + tr(inv(A - S)))

# solve and return
JuMP.optimize!(model)

┌ Warning: Function tr automatically registered with 1 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :tr, 1, tr; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear /Users/biona001/.julia/packages/MathOptInterface/cl3eR/src/Nonlinear/operators.jl:370


LoadError: Unexpected array VariableRef[S[1,1] S[1,2] S[1,3] S[1,4] S[1,5]; S[1,2] S[2,2] S[2,3] S[2,4] S[2,5]; S[1,3] S[2,3] S[3,3] S[3,4] S[3,5]; S[1,4] S[2,4] S[3,4] S[4,4] S[4,5]; S[1,5] S[2,5] S[3,5] S[4,5] S[5,5]] in nonlinear expression. Nonlinear expressions may contain only scalar expressions.

In [9]:
using Hypatia, JuMP, LinearAlgebra
p = 5
x = randn(p, p)
A = Symmetric(x' * x)

# model and constraints constraints
model = Model(() -> Hypatia.Optimizer())
@variable(model, S[1:p, 1:p], PSD)
@constraint(model, A >= S, PSDCone())

# introduce R and P such that R ⪰ inv(S) and P ⪰ inv(A - S)
@variable(model, R[1:p, 1:p])
@variable(model, P[1:p, 1:p])
@constraint(model, R >= inv(S), PSDCone())
@constraint(model, P >= inv(A - S), PSDCone())

# objective
@objective(model, Min, tr(R) + tr(P))

# solve and return
JuMP.optimize!(model)

LoadError: MethodError: no method matching VariableRef(::AffExpr)
[0mClosest candidates are:
[0m  VariableRef(::Any, [91m::Any[39m) at ~/.julia/packages/JuMP/puvTM/src/variables.jl:203
[0m  VariableRef([91m::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex}}[39m) at ~/.julia/packages/JuMP/puvTM/src/variables.jl:378
[0m  VariableRef([91m::Model[39m) at ~/.julia/packages/JuMP/puvTM/src/variables.jl:352
[0m  ...

In [10]:
using Hypatia, JuMP
p = 5
x = randn(p, p)
A = Symmetric(x' * x)

model = Model(() -> Hypatia.Optimizer())
@variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
@variable(model, t)
@variable(model, u)

# constraints
@constraint(model, S in PSDCone())
@constraint(model, A - S in PSDCone())
@constraint(model, t ≥ tr(inv(S)))
@constraint(model, u ≥ tr(inv(A - S)))

# objective
@objective(model, Min, t + u)

# solve and return
JuMP.optimize!(model)

LoadError: MethodError: no method matching VariableRef(::AffExpr)
[0mClosest candidates are:
[0m  VariableRef(::Any, [91m::Any[39m) at ~/.julia/packages/JuMP/puvTM/src/variables.jl:203
[0m  VariableRef([91m::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex}}[39m) at ~/.julia/packages/JuMP/puvTM/src/variables.jl:378
[0m  VariableRef([91m::Model[39m) at ~/.julia/packages/JuMP/puvTM/src/variables.jl:352
[0m  ...

This uses the original MVR objective, but it is not clear how to rewrite the constraints using this objective. 

In [11]:
using Hypatia, JuMP, LinearAlgebra, BlockDiagonals
p = 5
x = randn(p, p)
A = Symmetric(x' * x)

# model and constraints constraints
model = Model(() -> Hypatia.Optimizer())
@variable(model, S[1:p, 1:p], PSD)
@constraint(model, A >= S, PSDCone())

X = [A       A - S;
     A - S   A    ]

# objective
@objective(model, Min, tr(X))

# solve and return
JuMP.optimize!(model)


 iter        p_obj        d_obj |  abs_gap    x_feas    z_feas |      tau       kap        mu | dir_res     prox  step     alpha
    0   3.9280e+01   3.2297e+01 | 1.00e+01  4.85e-01  3.34e-01 | 1.00e+00  1.00e+00  1.00e+00 |
    1   3.9280e+01   3.6671e+01 | 2.98e+00  1.93e-01  1.33e-01 | 7.55e-01  4.24e-01  3.00e-01 | 2.2e-16  5.1e-01  co-a  7.00e-01
    2   3.9280e+01   3.8872e+01 | 4.50e-01  2.74e-02  1.88e-02 | 7.97e-01  3.44e-02  4.34e-02 | 4.4e-16  7.0e-01  co-a  8.50e-01
    3   3.9280e+01   3.9276e+01 | 4.38e-03  2.67e-04  1.84e-04 | 8.18e-01  3.82e-04  4.27e-04 | 1.7e-16  9.0e-01  co-a  9.90e-01
    4   3.9280e+01   3.9279e+01 | 6.09e-04  3.84e-05  2.64e-05 | 8.54e-01  6.46e-05  6.04e-05 | 1.8e-14  4.8e-01  co-a  8.50e-01
    5   3.9280e+01   3.9280e+01 | 6.01e-05  3.57e-06  2.45e-06 | 9.19e-01  5.16e-06  5.90e-06 | 1.1e-14  9.4e-01  co-a  9.00e-01
    6   3.9280e+01   3.9280e+01 | 5.89e-07  3.52e-08  2.42e-08 | 9.32e-01  5.80e-08  5.85e-08 | 1.8e-15  3.5e-01  co-a  9.90e-01


In [5]:
using Hypatia, JuMP, LinearAlgebra
p = 5
x = randn(p, p)
A = Symmetric(x' * x)

# model and constraints constraints
model = Model(() -> Hypatia.Optimizer())
@variable(model, S[1:p, 1:p], PSD)
@constraint(model, A >= S, PSDCone())

# introduce X and Y
@variable(model, X[1:p, 1:p])
@variable(model, Y[1:p, 1:p])
@constraint(model, [X I; I S] in PSDCone())
@constraint(model, [Y I; I A-S] in PSDCone())

# objective
@objective(model, Min, tr(X) + tr(Y))

# solve and return
JuMP.optimize!(model)


 iter        p_obj        d_obj |  abs_gap    x_feas    z_feas |      tau       kap        mu | dir_res     prox  step     alpha
    0   2.1273e+01  -2.2047e+01 | 3.00e+01  3.94e-01  6.55e-01 | 1.00e+00  1.00e+00  1.00e+00 |
    1   1.3471e+01  -9.4483e+00 | 8.95e+00  2.14e-01  3.56e-01 | 5.52e-01  6.39e-01  3.00e-01 | 7.1e-15  6.2e-01  co-a  7.00e-01
    2   1.0967e+01  -6.3627e-01 | 3.60e+00  1.09e-01  1.82e-01 | 4.33e-01  2.91e-01  1.20e-01 | 1.8e-15  6.6e-01  co-a  6.00e-01
    3   1.2823e+01   6.3209e+00 | 1.44e+00  6.29e-02  1.05e-01 | 3.01e-01  1.73e-01  4.83e-02 | 1.8e-15  7.0e-01  co-a  6.00e-01
    4   1.5315e+01   1.0820e+01 | 7.15e-01  4.50e-02  7.48e-02 | 2.10e-01  1.19e-01  2.39e-02 | 2.6e-14  7.7e-01  co-a  5.00e-01
    5   1.9491e+01   1.6895e+01 | 2.81e-01  2.81e-02  4.67e-02 | 1.35e-01  7.60e-02  9.39e-03 | 3.0e-14  9.8e-01  co-a  6.00e-01
    6   2.3508e+01   2.1673e+01 | 1.38e-01  2.23e-02  3.70e-02 | 8.50e-02  5.69e-02  4.61e-03 | 4.2e-13  9.5e-01  co-a  5.00e-01


In [104]:
using Hypatia, JuMP, LinearAlgebra
p = 5
x = randn(p, p)
A = Symmetric(x' * x)

# model and constraints constraints
model = Model(() -> Hypatia.Optimizer())
@variable(model, S[1:p, 1:p], PSD)
set_start_value.(S, A)

5×5 Matrix{Nothing}:
 nothing  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  nothing

## Block MVR (using Convex.jl)

In [ ]:
using Convex

# function solve_SDP(Σ::AbstractMatrix)
#     svar = Variable(size(Σ, 1), Convex.Positive())
#     add_constraint!(svar, svar ≤ 1)
#     constraint = 2*Symmetric(Σ) - diagm(svar) in :SDP
#     problem = maximize(sum(svar), constraint)
#     solve!(problem, Hypatia.Optimizer; silent_solver=true)
#     s = clamp.(evaluate(svar), 0, 1) # make sure s_j ∈ (0, 1)
#     return s
# end

# this uses Convex.jl
function solve_group_MVR_single_block(
    Σ11::AbstractMatrix,
    A11::AbstractMatrix,
    A12::AbstractMatrix, 
    invD22::AbstractMatrix,
    m::Int,
    ub::AbstractMatrix; # this is upper bound, equals [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    )
    # needed constants
    invZYt = invD22 * A12'
    YinvZYt = A12 * invZYt
    
    # constraints
    p = size(Σ11, 1)
    svar = Semidefinite(p)
    add_constraint!(svar, svar ≤ 1)
    add_constraint!(svar, svar ≥ -1)
    constraint = ub - svar in :SDP
    
    # objective
    problem = minimize(
        m^2*tr(inv(S)) + tr(A11-S-YinvZYt) + tr(YinvZYt'*(A11-S-YinvZYt)*YinvZYt),
        constraint
    )

    # solve and return
    JuMP.optimize!(model)
    success = Knockoffs.check_model_solution(model)
    return JuMP.value.(S), success
end


In [18]:
p = size(Σ11, 1)
S = Semidefinite(p)
add_constraint!(S, svar ≤ 1)
add_constraint!(S, svar ≥ -1)
S

Variable
size: (5, 5)
sign: real
vexity: affine
id: 134…327

In [19]:
S.constraints

3-element Vector{Constraint}:
 sdp constraint (affine)
└─ 5×5 real variable (id: 134…327)
 <= constraint (affine)
├─ 5×5 real variable (id: 355…811)
└─ 1
 >= constraint (affine)
├─ 5×5 real variable (id: 355…811)
└─ -1

In [20]:
constraint = ub - S in :SDP

sdp constraint (affine)
└─ + (affine; real)
   ├─ 5×5 Symmetric{Float64, Matrix{Float64}}
   └─ - (affine; real)
      └─ 5×5 real variable (id: 134…327)

In [22]:
problem = minimize(
    m^2*tr(inv(S)) + tr(A-S-YinvZYt) + tr(invZYt'*(A11-S-YinvZYt)*YinvZYt),
    constraint
)

LoadError: Cannot add expressions of sizes (50, 50) and (5, 5)

In [24]:
A

50×50 Matrix{Float64}:
  2.0        -0.271149   -0.032879    …  -0.0947729    0.0801876
 -0.271149    2.0        -0.0243168      -0.0314542   -0.161222
 -0.032879   -0.0243168   2.0            -0.0182271   -0.101857
 -0.0761243  -0.0934999  -0.141387        0.316618     0.103354
 -0.020243    0.164665   -0.00873287     -0.00705244  -0.0531738
 -0.145071   -0.0814823  -0.119608    …   0.0537927   -0.200701
 -0.106396   -0.0385128  -0.0083662       0.0765058   -0.282234
  0.0704449   0.0450692   0.0910859      -0.0721822    0.265512
  0.229754    0.143721   -0.0126377      -0.0309272    0.219629
  0.0354905  -0.168119    0.115841        0.00548069  -0.0542383
  0.196919   -0.0718144  -0.0108042   …   0.0735863   -0.262046
 -0.135652    0.0582198  -0.0353113      -0.0161846   -0.00994014
 -0.174641    0.137293   -0.110705        0.010365    -0.0552382
  ⋮                                   ⋱               
 -0.124399    0.102943    0.207536        0.00364047  -0.0590465
  0.0258755  -0.041

## Use @objective with slack variables or NLobjective directly?

In [11]:
function slack_SDP_single_block(
    Σ11::AbstractMatrix,
    ub::AbstractMatrix;  # this is upper bound, equals [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false, iter_limit=100) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    # slack variables to handle absolute value in obj 
    @variable(model, U[1:p, 1:p], Symmetric)
    for i in 1:p, j in i:p
        @constraint(model, Σ11[i, j] - S[i, j] ≤ U[i, j])
        @constraint(model, -U[i, j] ≤ Σ11[i, j] - S[i, j])
    end
    @objective(model, Min, sum(U))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

using SCS
function NLObj_SDP_single_block(
    Σ11::AbstractMatrix,
    ub::AbstractMatrix;  # this is upper bound, equals [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=SCS.Optimizer() # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    @NLobjective(model, Min, sum(abs(Σ11[i, j] - S[i, j]) for i in 1:p, j in 1:p))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

NLObj_SDP_single_block (generic function with 1 method)

In [12]:
m = 1

# initialize with equicorrelated solution
# Sequi, γ = solve_s_group(Σ, groups, :equi)
Sequi = zeros(p, p)

# form constraints for block 1
Σ11 = Σ[1:5, 1:5]
A = (m+1)/m * Σ
D = A - Sequi
A11 = @view(A[1:5, 1:5])
D12 = @view(D[1:5, 6:end])
D22 = @view(D[6:end, 6:end])
ub = A11 - D12 * inv(D22) * D12'

# solve 
@time slack = slack_SDP_single_block(Σ11, ub)
@time nl = NLObj_SDP_single_block(Σ11, ub);

  0.209440 seconds (458.88 k allocations: 24.826 MiB, 57.65% compilation time)


LoadError: The solver does not support nonlinear problems (i.e., NLobjective and NLconstraint).

In [22]:
using JuMP
using MosekTools

model = Model(Mosek.Optimizer)

@variable(model, 2.5 ≤ z1 ≤ 5.0)
@variable(model, -1.0 <= z2 <= 1.0)
@objective(model, Min, abs(z1+5.0) + abs(z2-3.0))

LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.GreaterThan{Float64} are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

Try block descent

In [6]:
@time Ssdp, _ = solve_s_group(
    Σ, groups, :sdp,
    m = 1, 
    verbose=true,
    niter=5
);

Init obj = 651.4494943754169
Iter 1 δ = 0.9536228199702621, obj = 430.76840338555894


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 2 δ = 0.826901097351207, obj = 426.18848412004144


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 3 δ = 0.029548257620773677, obj = 425.975056006773
Iter 4 δ = 0.5597317692795303, obj = 419.2123948332009


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 5 δ = 0.035389868543257264, obj = 419.0889051776713
 44.680826 seconds (127.32 M allocations: 7.957 GiB, 3.62% gc time, 77.06% compilation time)


In [3]:
@time Sequi, _ = solve_s_group(
    Σ, groups, :equi,
    m = 1, 
);

  0.139828 seconds (439.08 k allocations: 24.683 MiB, 86.85% compilation time)


In [8]:
@time Ssdp_subopt, _ = solve_s_group(
    Σ, groups, :sdp_subopt,
    m = 1, 
);

  1.880804 seconds (856.86 k allocations: 91.592 MiB, 7.57% gc time, 35.69% compilation time)


In [4]:
@time Sme, _ = solve_s_group(
    Σ, groups, :maxent,
    m = 1, 
    verbose=true
);

initial obj = -1001.2975818995944
Iter 1: obj = -676.9797433696642, δ = 0.7148761215068528, t1 = 0.14, t2 = 0.04, t3 = 0.0
Iter 2: obj = -667.02018374056, δ = 0.1389593291910438, t1 = 0.15, t2 = 0.05, t3 = 0.0
Iter 3: obj = -665.1059850352791, δ = 0.021527165678117164, t1 = 0.16, t2 = 0.07, t3 = 0.0
Iter 4: obj = -664.2229660971566, δ = 0.00986453850330372, t1 = 0.17, t2 = 0.08, t3 = 0.0
  0.553342 seconds (1.08 M allocations: 58.418 MiB, 5.56% gc time, 79.26% compilation time)


In [9]:
sum(Sequi), sum(Ssdp), sum(Sme), sum(Ssdp_subopt)

(31.681698156089773, 312.2139302866235, 66.42467742735388, 223.5460638065474)

In [11]:
Ssdp

200×200 Matrix{Float64}:
 1.0       0.559985  0.551354  0.472459  …  0.0        0.0       0.0
 0.559985  1.0       0.983603  0.842856     0.0        0.0       0.0
 0.551354  0.983603  1.0       0.856051     0.0        0.0       0.0
 0.472459  0.842856  0.856051  1.0          0.0        0.0       0.0
 0.37607   0.670901  0.681403  0.795189     0.0        0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 ⋮                                       ⋱                       
 0.0       0

In [10]:
Ssdp_subopt

200×200 Matrix{Float64}:
 1.0       0.559985  0.551354  0.472459  …  0.0       0.0       0.0
 0.559985  1.0       0.983603  0.842856     0.0       0.0       0.0
 0.551354  0.983603  1.0       0.856051     0.0       0.0       0.0
 0.472459  0.842856  0.856051  1.0          0.0       0.0       0.0
 0.37607   0.670901  0.681403  0.795189     0.0       0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 ⋮                                       ⋱                      
 0.0       0.0       0.0  

In [222]:
[vec(Sequi) vec(Ssdp) vec(Sme)]

40000×3 Matrix{Float64}:
 0.0626931  1.0       0.555897
 0.0417102  0.665309  0.184368
 0.0302182  0.482002  0.0209709
 0.0288381  0.459989  0.00883674
 0.0247707  0.39511   0.00441798
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 ⋮                    
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0130953  0.39511   0.000925487
 0.0167721  0.593875  0.00262455
 0.0186497  0.819727  0.00893611
 0.0515229  0.858956  0.169722
 0.0626931  1.0       0.32336

What if we run group knockoffs when all groups are singletons?

In [127]:
groups = collect(1:p)

# group SDP with single groups
@time Ssdp, _ = solve_s_group(
    Σ, groups, :sdp_block,
    m = 1, 
    verbose=true,
    niter=5
)

# non-grouped SDP
@time ssdp = solve_s(
    Symmetric(Σ), :sdp,
    m = 1, 
    verbose=true,
);

g = 1
A11 = [2.0;;]
size(A11) = (1, 1)


LoadError: UndefVarError: fff not defined

Here Ssdp should be diagonal matrix.

In [87]:
Ssdp

200×200 Matrix{Float64}:
 1.0  0.0  0.0  0.0       0.0      0.0       …  0.0       0.0       0.0
 0.0  1.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  1.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.995473  0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.63106  0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.554254  …  0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0       …  0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 ⋮                                 ⋮   

In [86]:
[ssdp diag(Ssdp)]

200×2 Matrix{Float64}:
 1.0         1.0
 0.468434    1.0
 0.86833     1.0
 0.914276    0.995473
 3.36244e-8  0.63106
 0.5571      0.554254
 4.10019e-8  0.298628
 0.236487    0.265939
 0.0911048   0.198897
 0.0657967   0.186742
 0.198103    0.351992
 0.29984     0.611173
 0.694426    0.84924
 ⋮           
 0.0318133   0.047177
 0.011571    0.0522678
 0.599472    0.822529
 0.412487    0.965399
 0.819467    0.90014
 1.32504e-8  0.398157
 0.354499    0.437457
 0.403364    0.483912
 1.07897e-8  0.136476
 0.115996    0.112647
 1.02813e-8  0.181416
 0.376427    0.371388

In [89]:
eigmin(2Σ - Diagonal(ssdp))

-4.075475130804836e-9

In [91]:
eigmin(2Σ - Ssdp)

-0.3749872292309029

## How to permute things

In [224]:
S1 = reshape(1:16, 4, 4) |> Matrix
S2 = reshape(1:9, 3, 3) |> Matrix
S3 = reshape(1:4, 2, 2) |> Matrix
S = BlockDiagonal([S1, S2, S3])

9×9 BlockDiagonal{Int64, Matrix{Int64}}:
 1  5   9  13  0  0  0  0  0
 2  6  10  14  0  0  0  0  0
 3  7  11  15  0  0  0  0  0
 4  8  12  16  0  0  0  0  0
 0  0   0   0  1  4  7  0  0
 0  0   0   0  2  5  8  0  0
 0  0   0   0  3  6  9  0  0
 0  0   0   0  0  0  0  1  3
 0  0   0   0  0  0  0  2  4

In [233]:
perm = collect(1:9)
g = 2
offset = 7
cur_idx = offset + 1:offset + g
for i in 1:offset
    perm[g+i] = i
end
perm[1:g] .= cur_idx
perm

9-element Vector{Int64}:
 8
 9
 1
 2
 3
 4
 5
 6
 7

In [235]:
Spermuted = S[perm, perm]

9×9 Matrix{Int64}:
 1  3  0  0   0   0  0  0  0
 2  4  0  0   0   0  0  0  0
 0  0  1  5   9  13  0  0  0
 0  0  2  6  10  14  0  0  0
 0  0  3  7  11  15  0  0  0
 0  0  4  8  12  16  0  0  0
 0  0  0  0   0   0  1  4  7
 0  0  0  0   0   0  2  5  8
 0  0  0  0   0   0  3  6  9

In [236]:
iperm = invperm(perm)
Spermuted[iperm, iperm]

9×9 Matrix{Int64}:
 1  5   9  13  0  0  0  0  0
 2  6  10  14  0  0  0  0  0
 3  7  11  15  0  0  0  0  0
 4  8  12  16  0  0  0  0  0
 0  0   0   0  1  4  7  0  0
 0  0   0   0  2  5  8  0  0
 0  0   0   0  3  6  9  0  0
 0  0   0   0  0  0  0  1  3
 0  0   0   0  0  0  0  2  4

## Solving Convex problems

Try solving first block

In [3]:
#
# test problem
#
p = 15
group_sizes = [5 for i in 1:div(p, 5)] # each group has 5 variables
groups = vcat([i*ones(g) for (i, g) in enumerate(group_sizes)]...) |> Vector{Int}
# Σ = simulate_block_covariance(groups, 0.75, 0.25)
# Σ = Matrix(SymmetricToeplitz(0.4.^(0:(p-1)))) # true covariance matrix
# Σ = Matrix(SymmetricToeplitz((-0.4).^(0:(p-1)))) # true covariance matrix
Σ = simulate_AR1(p, a=3, b=1) # true covariance matrix

"""
    solve_full_SDP
"""
function solve_full_SDP(
    Σ11::AbstractMatrix, # correlation matrix
    ub::AbstractMatrix; # this is [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    # slack variables to handle absolute value in obj 
    @variable(model, U[1:p, 1:p], Symmetric)
    for i in 1:p, j in i:p
        @constraint(model, Σ11[i, j] - S[i, j] ≤ U[i, j])
        @constraint(model, -U[i, j] ≤ Σ11[i, j] - S[i, j])
    end
#     @constraint(model, U in PSDCone())      #### is this constraint needed????
    @objective(model, Min, sum(U))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S), objective_value(model)
end

function solve_full_SDP_old(
    Σ11::AbstractMatrix, # correlation matrix
    ub::AbstractMatrix; # this is [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, 0 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    @objective(model, Min, sum(Σ11 - S))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

solve_full_SDP_old (generic function with 1 method)

In [4]:
m = 1

# initialize with equicorrelated solution
# Sequi, γ = solve_s_group(Σ, groups, :equi)
Sequi = zeros(p, p)

# form constraints for block 1
Σ11 = Σ[1:5, 1:5]
A = (m+1)/m * Σ
D = A - Sequi
A11 = @view(A[1:5, 1:5])
D12 = @view(D[1:5, 6:end])
D22 = @view(D[6:end, 6:end])
ub = A11 - D12 * inv(D22) * D12'

# solve 
@time S1_new, obj = solve_full_SDP(Σ11, ub)
@time S1_old = solve_full_SDP_old(Σ11, ub);

 38.529136 seconds (139.35 M allocations: 8.343 GiB, 4.92% gc time, 99.33% compilation time)
  0.162872 seconds (511.00 k allocations: 30.119 MiB, 13.71% gc time, 91.17% compilation time)


In [163]:
obj, sum(abs.(Σ11 - S1_new))

(2.696897785289176e-9, 9.016139224105046e-10)

In [160]:
eigmin(S1_new)

0.0790755108995826

In [161]:
eigmin(ub)

0.1581510007508361

Lets eyeball the result first

In [94]:
S1_new

5×5 Matrix{Float64}:
 1.0     0.4    0.16  0.064  0.0256
 0.4     1.0    0.4   0.16   0.064
 0.16    0.4    1.0   0.4    0.16
 0.064   0.16   0.4   1.0    0.4
 0.0256  0.064  0.16  0.4    1.0

In [95]:
S1_old

5×5 Matrix{Float64}:
 0.789886  0.747998  0.769968  0.644723  0.319591
 0.747998  0.962284  0.938245  0.82901   0.502877
 0.769968  0.938245  0.982537  0.854499  0.525783
 0.644723  0.82901   0.854499  0.879502  0.417846
 0.319591  0.502877  0.525783  0.417846  0.477052

In [96]:
sum(abs.(S1_new)), sum(abs.(S1_old))

(9.467199998341476, 17.192337396303046)

Check objective

In [162]:
sum(abs.(Σ11 - S1_new)), sum(abs.(Σ11 - S1_old))

(9.016139224105046e-10, 8.45111995244404)

Check constraints

In [98]:
isposdef(S1_old), isposdef(S1_new)

(true, true)

In [99]:
eigvals(ub - S1_old)

5-element Vector{Float64}:
 7.309755484523514e-9
 0.7520353697688693
 0.9080664264148712
 1.4359163255019285
 2.0506742273768

In [100]:
eigvals(ub - S1_new)

5-element Vector{Float64}:
 0.32129497088496595
 0.4766120211625212
 0.643214501447116
 1.0285108585981437
 1.7683208812336677

### Singleton groups?

In [105]:
groups[2:5] .= 2
groups

15-element Vector{Int64}:
 1
 2
 2
 2
 2
 2
 2
 2
 2
 2
 3
 3
 3
 3
 3

In [132]:
m = 1

# initialize with equicorrelated solution
Sequi, γ = solve_s_group(Σ, groups, :equi)

# form constraints for block 1
g = 1
Σ11 = @view(Σ[1, 1])
A = (m+1)/m * Σ
D = A - Sequi
Σ11 = @view(Σ[1:g, 1:g])
A11 = @view(A[1:g, 1:g])
D12 = @view(D[1:g, g + 1:end])
D21 = @view(D[g + 1:end, 1:g])
D22 = @view(D[g + 1:end, g + 1:end])
ub = Symmetric(A11 - D21' * inv(D22) * D21)

# solve 
@time S1_new, U = solve_full_SDP(Σ11, ub)
# @time S1_old = solve_full_SDP_old(Σ11, ub);

  0.066523 seconds (118.17 k allocations: 6.466 MiB, 89.63% compilation time)


In [134]:
S1_new, ub

([0.8644309427462861;;], [0.8644309426414065;;])

## Woodbury updates

Suppose we have 

\begin{align*}
    S = 
    \begin{bmatrix}
        S^{(1)} & & \\
        & S^{(2)} & \\
        & & S^{(3)}
    \end{bmatrix}
\end{align*}
And we want to udpate $S^{(1)}$

$$(A + UC)^{-1} = A^{-1} - A^{-1}U(I + VA^{-1}U)^{-1}VA^{-1}$$

In [1]:
using WoodburyMatrices

In [ ]:
A = 